# Khai báo thư viện

In [1]:
import pandas as pd
import re
from underthesea import word_tokenize

In [2]:
# Đọc dữ liệu
df = pd.read_csv('tuyensinh.csv')
def clean_comment(text):
     # Kiểm tra nếu text không xác định
    if pd.isna(text):
        return ""
    # Chuyển đổi chữ hoa thành chữ thường
    text = text.lower()
    # bỏ link
    text = re.sub(r"http\S+|www\S+", "", text)
    # bỏ mention và hashtag
    text = re.sub(r"@\w+|#\w+", " ", text)
    # Loại bỏ email
    text = re.sub(r'\S+@\S+', ' ', text)
    # Loại bỏ số điện thoại
    text = re.sub(r'\b\d{10,11}\b', ' ', text)
    # chỉ giữ lại chữ cái, khoảng trắng, số và dấu câu cơ bản
    text = re.sub(r"[^a-zA-ZÀ-ỹà-ỹ0-9\s. /]", " ", text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip() 
    # Loại bỏ các ký tự lặp lại hơn 2 lần liên tiếp
    text = re.sub(r'(.)\1+', r'\1\1', text)
    # Ánh xạ các từ viết tắt thông dụng
    replace_dict = {
        'k': 'không','ko': 'không','kh': 'không','hok': 'không','j': 'gì','r': 'rồi','dc': 'được','đc': 'được','vs': 'với','cx': 'cũng','trc': 'trước','ds': 'danh sách',
        'ch': 'chưa','v': 'vậy','z': 'vậy','cb': 'các bạn','lg': 'lượng','in4': 'thông tin','ntn': 'như thế nào','bn': 'bạn','bh': 'bao giờ','trg': 'trường','lmj': 'làm gì',
        'sv': 'sinh viên','ad': 'admin','mn': 'mọi người','kt': 'kiểm tra','e': 'em','a': 'anh','ae': 'anh em','c': 'chị','t': 'tôi','b': 'bạn','mk': 'mình','rùi': 'rồi',
        'đh': 'đại học','thpt': 'trung học phổ thông','ng': 'người','h': 'giờ','ptien': 'phương tiện','fb': 'facebook','tsv': 'tân sinh viên','tb': 'thông báo','ktx': 'kí túc xá',
        'clb': 'câu lạc bộ','cntt': 'công nghệ thông tin','chx':'chưa','hs': 'hồ sơ','hb': 'học bạ','clc': 'chất lượng cao','kq': 'kết quả','ny': 'người yêu','đk': 'đăng ký',
        'tmdt': 'thương mại điện tử','m': 'mày','bt': 'biết','đzai': 'đẹp trai','ktpm': 'kỹ thuật phần mềm','tyeu': 'tình yêu','dt vt': 'điện tử viễn thông','nv': 'nguyện vọng',
        'ktđ đt': 'kỹ thuật điện điện tử','tkđh': 'thiết kế đồ họa','tkdh': 'thiết kế đồ họa','cn' : 'công nghệ','xh': 'xã hội','đhoc': 'đại học','nm': 'nhưng mà','cmt': 'comment',
        'khmt':'khoa học máy tính','tnv': 'tình nguyện viên','tv': 'thành viên','tgia': 'tham gia','ngyeu': 'người yêu','cbi': 'chuẩn bị','nhg': 'nhưng','thg':'thằng','đg':'đường',
        'thk':'thằng','kte':'kinh tế','ktmt':'kỹ thuật máy tính','nthe':'như thế','cnoto':'công nghệ ô tô','tr': 'trường','dki': 'đăng ký','đki': 'đăng ký','cmay':'chúng mày',
        'gét gô': 'get go','omg': 'oh my god','yc': 'yêu cầu','hk': 'học'
    }
    for key, value in replace_dict.items():
        pattern = r"\b" + re.escape(key) + r"\b"
        text = re.sub(pattern, value, text)
    
    return text

# Áp dụng hàm làm sạch vào cột 'comment'
df['comment_cleaned'] = df['comment'].apply(clean_comment)
# Loại bỏ dòng trùng lặp
df = df.drop_duplicates(subset=['full_name', 'comment'])
# Xóa các dòng có comment_cleaned trống
df = df[df['comment_cleaned'].str.strip() != ""]
# Loại bỏ các bình luận spam hoặc quảng cáo 
spam_keywords = [
    'tuyển dụng', 'học bổng', 'model', 'casting', 'tuyển tình nguyện viên',
    'laptop', 'phòng trọ', 'cho thuê', 'bán', 'mua', 'giảm giá', 'khuyến mãi',
    'liên hệ', 'zalo', 'model','đài loan', 'học bổng toàn phần', 'ưu đãi', 'hotline','event',
] 
def is_spam(text):
    if not text or pd.isna(text):
        return False
    for kw in spam_keywords:
        if kw in text.lower():
            return True
    return False
df = df[~df['comment_cleaned'].apply(is_spam)]

# Tách từ bằng underthesea
def tokenize_text(text):
    if text.strip():
        text = word_tokenize(text, format="text")
    return text
# df['comment_cleaned'] = df['comment_cleaned'].apply(tokenize_text)

# Xuất ra file CSV sau khi làm sạch dữ liệu
df.to_csv("tuyensinh_cleaned.csv", index=False, encoding="utf-8-sig")
# Xem kết quả
# df[1:50]